In [16]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [17]:
words = open('names.txt', 'r').read().splitlines()

In [18]:
all_characters = sorted(set(''.join(words)))
stoi = {char: index + 1 for index, char in enumerate(all_characters)}
stoi['.'] = 0
itos = { char: index for index, char in stoi.items()}

In [19]:
block_size = 3
X, Y = [], []

for word in words[:10]:
    context = [0] * block_size
    for char in word + '.':
        ichar = stoi[char]
        X.append(context)
        Y.append(ichar)
        # print(''.join(itos[i] for i in context), '-->', char)
        context = context[1:] + [ichar]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [20]:
g = torch.Generator().manual_seed(3498561551) # for reproducibility
C = torch.randn((27, 2), requires_grad=True, generator=g)
W1 = torch.randn((6, 10), requires_grad=True, generator=g)
b1 = torch.randn(10, requires_grad=True, generator=g)
W2 = torch.randn((10, 27), requires_grad=True, generator=g)
b2 = torch.randn(27, requires_grad=True, generator=g)
parameters = [C, W1, b1, W2, b2]

In [23]:
embedding = C[X]
x = torch.tanh(embedding.view(X.shape[0], 6) @ W1 + b1)
logits = x @ W2 + b2
loss = F.cross_entropy(logits, Y)

for p in parameters:
    p.grad = None
loss.backward()

for p in parameters:
    p.data += -0.1 * p.grad
print(loss.item())


5.911178112030029
